In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve, make_scorer, f1_score, roc_auc_score
from sklearn import preprocessing
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, LeaveOneGroupOut, PredefinedSplit, GridSearchCV
import matplotlib.pyplot as plt
import os
import json

%matplotlib inline

In [2]:
def concat_dataframes(path, df_type):
    dfs_list = []
    dfs_rows_len_list = []
    
    for user in os.listdir(path):
        for file in os.listdir(os.path.join(path, user)):
            if file.find(df_type) != -1:
                df = pd.read_csv(os.path.join(path, user, file))
                
                if df_type != 'broadcasts':
                    df = df.drop(["timestamp"], axis=1)
#                 df = (df - df.min()) / (df.max() - df.min())
                
                df["user"] = int(user.split('_')[1])
                
                dfs_list.append(df)
    
    return pd.concat(dfs_list, ignore_index=True)


def drop_bad_rows(df, z = 3):
    bad_rows = set()
    for col in df.columns:
        if col != "user":
            for user in df.user.unique():
                for x in list(df.loc[df.user == user, :][np.abs(stats.zscore(df.loc[df.user == user, col])) > z].index):
                    bad_rows.add(x)

            for x in list(df[col][np.abs(stats.zscore(df[col])) > z].index):
                bad_rows.add(x)

    df = df.drop(list(bad_rows), axis=0)

    return df


def drop_bad_cols(df, z = 3, allowed_proportion = 0.1):
    bad_cols = set()
    for col in df.columns:
        if col != "user":
            if df[df[col] != df[col].mean()].shape[0] < allowed_proportion * df.shape[0]:
                bad_cols.add(col)

            for user in df.user.unique():
                if df.loc[df.user == user, :][df.loc[df.user == user, col] != df.loc[df.user == user, col].mean()].shape[0] < allowed_proportion * df.loc[df.user == user, :].shape[0]:
                    bad_cols.add(col)

                elif np.sum(np.abs(stats.zscore(df.loc[df.user == user, col])) < z) < (1 - allowed_proportion) * df.loc[df.user == user, col].shape[0]:
                    bad_cols.add(col)

    df = df.drop(bad_cols, axis=1)
    return df, list(bad_cols)
    
    
def extract_delayed_user(df, user_label):
    df_user = df[df["user"] == user_label]
    df = df[df["user"] != user_label]
    return df_user, df


def split_users_into_two_classes(df, valid_user_label):
    df.loc[df["user"] != valid_user_label, "user"] = 0
    df.loc[df["user"] == valid_user_label, "user"] = 1
    return df  


def get_cv_split(X, y, group_labels, valid_user_label):
    predefined_split_array = np.zeros(group_labels.shape[0])
    i = 0
    test_array = [x for x in range(group_labels.shape[0])]
    for test, _ in LeaveOneGroupOut().split(X, y, group_labels):
        diff = np.setdiff1d(test_array, test)
        if np.all(group_labels[diff[0] : diff[-1]] == valid_user_label) is np.bool_(True):
            for sample in diff:
                predefined_split_array[sample] = -1
        else:
            for sample in diff:
                predefined_split_array[sample] = i
            i += 1
    return predefined_split_array


def generate_train_dataset(df, user, ex_user, is_SVM = False):
    df_ = df.copy()

    df_for_test = []

    df__ = df_[df_.labels == ex_user].copy()
    df_for_test.append(df__)
    df_ = df_.drop(df__.index, axis=0)

    for user_ in df_.labels.unique():
        if user_ != ex_user:
            test_size = int((0.25 * df_[df_.labels == user_].shape[0]) - 1)
            df__ = df_[df_.labels == user_].sample(test_size).copy()
            df_for_test.append(df__)
            df_ = df_.drop(df__.index, axis=0)

    df_ = split_users_into_two_classes(df_.copy(), user)
          
    if is_SVM:    
        df_.loc[df_.user == 0, 'user'] = -1

    df_ = df_.drop("labels", axis=1)

    dataset = df_.to_numpy().copy()
    np.random.shuffle(dataset)

    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    return X, y, df_for_test


def generate_test_dataset(df_list, user, ex_user, is_SVM = False):
    test_df = pd.concat(df_list)

    valid_user_in_test_count = test_df[test_df.labels == user].shape[0]
    ex_user_in_test_count = test_df[test_df.labels == ex_user].shape[0]
    others_in_test_count = [test_df[test_df.labels == x].shape[0]
                            for x in test_df.labels.unique() if x != user and x != ex_user]

    others_test_count = sum(others_in_test_count)
    part_size = min(valid_user_in_test_count, ex_user_in_test_count)
    if others_test_count <= min(valid_user_in_test_count, ex_user_in_test_count):
        part_size = others_test_count    
        
    new_df_parts = []    

    new_df_parts.append(test_df[test_df.labels == user].sample(part_size).copy())
    new_df_parts.append(test_df[test_df.labels == ex_user].sample(part_size).copy())
    new_df_parts.append(test_df[~test_df.labels.isin([user, ex_user])].sample(part_size).copy())
    
    test_df = pd.concat(new_df_parts)
    
    test_df.loc[test_df.labels == user, "user"] = 1
    if is_SVM:
        test_df.loc[test_df.labels != user, "user"] = -1
    else:
        test_df.loc[test_df.labels != user, "user"] = 0

    print("True: ", test_df[test_df.user == 1].shape)
    print("Shape: ", test_df.shape)
    for x in test_df.labels.unique():
        print("Count ", x, ": ", test_df[test_df.labels == x].shape)

    test_df = test_df.drop("labels", axis=1)

    test_dataset = test_df.to_numpy().copy()
    X_test = test_dataset[:, :-1].copy()
    y_test = test_dataset[:, -1].copy()

    return X_test, y_test


def prepare_dataset(df, user, is_SVM=False):
    df_ = split_users_into_two_classes(df.copy(), user)
    
    group_labels = df_.labels.to_numpy().copy()
    df_ = df_.drop('labels', axis=1)
    
    if is_SVM:
        df_.loc[df_.user == 0, 'user'] = -1
    
    dataset = df_.to_numpy().copy()
    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    return X, y, group_labels


def create_file_for_results(data_type):
    res_folder = '.\\_results'
    if os.path.exists(res_folder) is False:
        os.makedirs(res_folder)
    
    file = os.path.join(res_folder, data_type + '_results.json')
    if os.path.exists(file) is False:
        with open(file, 'w') as f:
            json.dump({'stub': None}, f)
        
    return file    


def update_file_with_results(file_path, results_dict):
    import collections.abc

    def update(d, u):
        for k, v in u.items():
            if isinstance(v, collections.abc.Mapping):
                d[k] = update(d.get(k, {}), v)
            else:
                d[k] = v
        return d
    
    with open(file_path, 'r') as f:
        res = json.load(f)
    
    res = update(res, results_dict)
    
    with open(file_path, 'w') as f:
        json.dump(res, f, sort_keys=True, indent=2)
        
        
def get_dict_with_results(json_path):
    with open(json_path, 'r') as f:
        res = json.load(f)
    return res    


def get_dataframe(path, data_type, window_type, window_size):
    return concat_dataframes(os.path.join(path, window_type, window_size), data_type), create_file_for_results(data_type)


def drop_corr_columns(df, corr_coef):
    corr_matrix = df.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    corr_cols = [column for column in upper_tri.columns if any(abs(upper_tri[column]) > corr_coef) and column != "user"]
    return df.drop(corr_cols, axis=1), corr_cols


def process_train_df(df, features, corr = 0.7, z = 3, prop = 0.1): 
    df = df.drop(df.columns.difference(features), axis=1)
    df = df.dropna(how='all')
    df = df.fillna(0)
    
    if 'count_mean' in df.columns:
        df = df[df.count_mean != 0]
    
    df = drop_bad_rows(df, z)
    df, dropped_cols_1 = drop_bad_cols(df, z, prop)
    df, dropped_cols_2 = drop_corr_columns(df, corr)
    
    return df, dropped_cols_1 + dropped_cols_2

In [3]:
def model_cross_validation(results_file, model, df, model_tag, df_type, window_type, window_size, is_SVM = False):
    for user in df.labels.unique():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")

        X, y, group_labels = prepare_dataset(df, user, is_SVM)

        cv_split = PredefinedSplit(test_fold=get_cv_split(X, y, group_labels, user))
        scoring = ('accuracy')

        cv_results = cross_validate(model, X, y, scoring=scoring, cv=cv_split, n_jobs=-1)
        accuracy = cv_results['test_score']
        
        results = {
            df_type: {
                window_type: {
                    window_size: {
                        model_tag: {
                            "cross_validation": {
                                "valid_user": {
                                    str(user): {
                                        "accuracy": accuracy.tolist()
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
        
        update_file_with_results(results_file, results)

        print("CV accuracy list: ", accuracy)
        print("CV mean accuracy: ", np.mean(accuracy))
        print("CV min accuracy: ", np.min(accuracy))
        print("CV max accuracy: ", np.max(accuracy))

        print("--------------------------------------------------------------------------------")
        
        
def model_final_validation(results_file, model, df, model_tag, df_type, window_type, window_size, is_SVM = False):
    for user in df.labels.unique():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")
        for ex_user in df.labels.unique():
            if ex_user != user:
                X, y, df_for_test = generate_train_dataset(df, user, ex_user, is_SVM)

                model.fit(X, y)

                X_test, y_test = generate_test_dataset(df_for_test, user, ex_user, is_SVM)

                predict = model.predict(X_test)
#                 if is_SVM:
#                     proba = model.decision_function(X_test)
#                 else:
                proba = model.predict_proba(X_test)

                results = {
                    df_type: {
                        window_type: {
                            window_size: {
                                model_tag: {
                                    "final_validation": {
                                        "valid_user": {
                                            str(user): {
                                                "extracted_user": {
                                                    str(ex_user): {
                                                        "test": y_test.tolist(),
                                                        "predict": predict.tolist(),
                                                        "proba": proba.tolist()
                                                    }
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }

                update_file_with_results(results_file, results)

                print("Valid user = ", user, ", Extracted user = ", ex_user, "accuracy = ", accuracy_score(y_test, predict))
                print("--------------------------------------------------------------------------------")     

### Learning settings
### ***

In [4]:
DATA_PATH = '..\\..\\scripts\\_features_all'

DATA_TYPE = "bt"

WINDOW_TYPE = "rolling"
WINDOW_SIZE = "60s"

In [5]:
DATA_TYPES = ['wifi', 'bt', 'location']
WINDOW_TYPES = ['rolling', 'sampling']
# 60s, 90s, 120s
# WINDOWS = ['5s', '10s', '30s', '60s', '90s', '120s', '240s', '600s']
WINDOWS = ['10s']

In [6]:
catboost_params = {
    'iterations': 100,
    'depth': 6,
    'loss_function': 'Logloss',
    'l2_leaf_reg': 1,
    'leaf_estimation_iterations': 5,
    'logging_level': 'Silent'
}

randomforest_params = {
    'n_estimators': 100,
    'criterion': 'gini',
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'auto',
    'n_jobs': -1,
    'class_weight': 'balanced',
}

svc_params = {
    'C': 1,
    'kernel': 'rbf',
    'degree': 1,
    'gamma': 5,
    'probability': True
}

logreg_params = {
    'penalty': 'l2',
    'C': 0.01,
    'solver': 'newton-cg',
    'max_iter': 1000,
    'n_jobs': -1
}

MODELS = [
    (CatBoostClassifier(**catboost_params), "CatBoost"),
    (RandomForestClassifier(**randomforest_params), "RandomForest"),
#     (SVC(**svc_params), "SVC"),
    (LogisticRegression(**logreg_params), "LogReg")
]

### ***

## Cross-validation

In [7]:
for data_type in DATA_TYPES:
    for wnd_type in WINDOW_TYPES:
        for wnd in WINDOWS:
            df, RESULTS_FILE = get_dataframe(DATA_PATH, data_type, wnd_type, wnd)
            features = df.columns.to_list()
            df, _ = process_train_df(df, features)
            df['labels'] = df['user']
            
            for model, tag in MODELS:
                print(data_type, wnd_type, wnd, tag)
                model_cross_validation(RESULTS_FILE, model, df, tag, data_type, wnd_type, wnd, is_SVM=tag=='SVC')

D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater


wifi rolling 10s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.72587828 0.75162549 0.93578309 0.63160392 0.15447154 1.
 0.39082969]
CV mean accuracy:  0.6557417167315992
CV min accuracy:  0.15447154471544716
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [0.97141707 0.95448635 0.98430253 0.99201202 1.         0.53481894
 0.99126638]
CV mean accuracy:  0.9183290409584991
CV min accuracy:  0.5348189415041783
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [0.99838969 0.96981692 0.99964324 0.99470104 0.99593496 1.
 0.99781659]
CV mean accuracy:  0.9937574932790583
CV min acc

CV accuracy list:  [0.99758454 0.96091044 1.         0.96432394 0.99881367 1.
 0.99126638]
CV mean accuracy:  0.987556994597585
CV min accuracy:  0.9609104403760514
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  8
--------------------------------------------------------------------------------
CV accuracy list:  [0.99738325 0.9995052  0.99869961 1.         0.99976273 1.
 0.99721448]
CV mean accuracy:  0.9989378965915295
CV min accuracy:  0.9972144846796658
CV max accuracy:  1.0
--------------------------------------------------------------------------------


D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater


wifi sampling 10s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.7041116  0.76273654 0.93558401 0.64507042 0.15936255 1.
 0.41624365]
CV mean accuracy:  0.6604441108322219
CV min accuracy:  0.1593625498007968
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [0.97670059 0.94614265 0.98486612 0.99654289 1.         0.63174603
 0.99746193]
CV mean accuracy:  0.9333514594550366
CV min accuracy:  0.6317460317460317
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [0.99839314 0.97503671 1.         0.99590269 1.         1.
 1.        ]
CV mean accuracy:  0.9956189348087685
CV min acc

...

D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


bt sampling 10s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [1.         0.99460432 0.56173025 0.93274583 0.94895449 0.87162162
 0.61702128]
CV mean accuracy:  0.8466682541765878
CV min accuracy:  0.5617302493241214
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [1.         0.99820144 1.         1.         1.         1.
 1.        ]
CV mean accuracy:  0.999743062692703
CV min accuracy:  0.9982014388489209
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [1.         0.53846154 0.78401922 0.88234385 1.         0.95945946
 1.        ]
CV mean accuracy:  0.8806120099347213
CV mi

CV accuracy list:  [1.         1.         0.99910072 0.99699609 0.97711812 0.99507995
 1.        ]
CV mean accuracy:  0.9954706978746616
CV min accuracy:  0.9771181199752629
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  8
--------------------------------------------------------------------------------
CV accuracy list:  [1. 1. 1. 1. 1. 1. 1.]
CV mean accuracy:  1.0
CV min accuracy:  1.0
CV max accuracy:  1.0
--------------------------------------------------------------------------------
location rolling 10s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.99934632 0.96504445 0.99693654 0.85825334 0.99993671 1.
 1.        ]
CV mean accuracy:  0.9742167658676889
CV min accuracy:  0.8582533419704376
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
---------------------------

CV accuracy list:  [0.98467967 0.99986926 0.99773994 0.99223195 0.75778481 0.99878751
 0.99861304]
CV mean accuracy:  0.9613865969806703
CV min accuracy:  0.7577848101265823
CV max accuracy:  0.9998692639560727
--------------------------------------------------------------------------------
Valid User:  6
--------------------------------------------------------------------------------
CV accuracy list:  [0.97051996 0.97006145 0.99894531 0.91881838 0.70013545 0.84389209
 0.99029126]
CV mean accuracy:  0.913237698900623
CV min accuracy:  0.7001354454979094
CV max accuracy:  0.9989453066144343
--------------------------------------------------------------------------------
Valid User:  7
--------------------------------------------------------------------------------
CV accuracy list:  [0.98003714 1.         0.99984933 0.96816193 0.96637418 0.97436709
 1.        ]
CV mean accuracy:  0.9841128095486109
CV min accuracy:  0.9663741829103115
CV max accuracy:  1.0
-----------------------------

CV accuracy list:  [0.75661376 0.89983931 0.90104487 0.9047619  0.94783574 0.99340659
 0.95454545]
CV mean accuracy:  0.9082925185228016
CV min accuracy:  0.7566137566137566
CV max accuracy:  0.9934065934065934
--------------------------------------------------------------------------------
Valid User:  4
--------------------------------------------------------------------------------
CV accuracy list:  [0.98015873 0.99678629 1.         0.96890988 0.91120977 0.97142857
 0.93636364]
CV mean accuracy:  0.9664081244343131
CV min accuracy:  0.9112097669256382
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  5
--------------------------------------------------------------------------------
CV accuracy list:  [0.98280423 0.99946438 1.         0.88076214 0.86237514 0.97142857
 1.        ]
CV mean accuracy:  0.956690637604854
CV min accuracy:  0.8623751387347391
CV max accuracy:  1.0
--------------------------------------------

## Final Validation

In [ ]:
for data_type in DATA_TYPES:
    for wnd_type in WINDOW_TYPES:
        for wnd in WINDOWS:
            df, RESULTS_FILE = get_dataframe(DATA_PATH, data_type, wnd_type, wnd)
            features = df.columns.to_list()
            df, _ = process_train_df(df, features)
            df['labels'] = df['user']
                
            for model, tag in MODELS:
                print(data_type, wnd_type, wnd, tag)
                model_final_validation(RESULTS_FILE, model, df, tag, data_type, wnd_type, wnd, is_SVM=tag=='SVC')

D:\Program Files\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in greater


wifi rolling 10s CatBoost
Valid User:  1
--------------------------------------------------------------------------------
True:  (1241, 9)
Shape:  (3723, 9)
Count  1 :  (1241, 9)
Count  2 :  (1241, 9)
Count  5 :  (917, 9)
Count  4 :  (211, 9)
Count  6 :  (18, 9)
Count  7 :  (25, 9)
Count  3 :  (42, 9)
Count  8 :  (28, 9)
Valid user =  1 , Extracted user =  2 accuracy =  0.8176201987644373
--------------------------------------------------------------------------------
True:  (769, 9)
Shape:  (2307, 9)
Count  1 :  (769, 9)
Count  3 :  (769, 9)
Count  5 :  (543, 9)
Count  2 :  (76, 9)
Count  4 :  (109, 9)
Count  8 :  (21, 9)
Count  6 :  (6, 9)
Count  7 :  (14, 9)
Valid user =  1 , Extracted user =  3 accuracy =  0.8266146510619853
--------------------------------------------------------------------------------
True:  (1241, 9)
Shape:  (3723, 9)
Count  1 :  (1241, 9)
Count  4 :  (1241, 9)
Count  5 :  (948, 9)
Count  6 :  (20, 9)
Count  3 :  (55, 9)
Count  7 :  (34, 9)
Count  2 :  (157, 9)

Valid user =  4 , Extracted user =  2 accuracy =  0.8116356700047688
--------------------------------------------------------------------------------
True:  (699, 9)
Shape:  (2097, 9)
Count  4 :  (699, 9)
Count  3 :  (699, 9)
Count  5 :  (412, 9)
Count  1 :  (173, 9)
Count  2 :  (86, 9)
Count  8 :  (17, 9)
Count  6 :  (3, 9)
Count  7 :  (8, 9)
Valid user =  4 , Extracted user =  3 accuracy =  0.8683834048640916
--------------------------------------------------------------------------------
True:  (699, 9)
Shape:  (2097, 9)
Count  4 :  (699, 9)
Count  5 :  (699, 9)
Count  1 :  (418, 9)
Count  7 :  (19, 9)
Count  2 :  (152, 9)
Count  3 :  (51, 9)
Count  8 :  (42, 9)
Count  6 :  (17, 9)
Valid user =  4 , Extracted user =  5 accuracy =  0.7949451597520267
--------------------------------------------------------------------------------
True:  (246, 9)
Shape:  (738, 9)
Count  4 :  (246, 9)
Count  6 :  (246, 9)
Count  8 :  (10, 9)
Count  1 :  (62, 9)
Count  7 :  (4, 9)
Count  5 :  (143, 9)
C

Valid user =  7 , Extracted user =  5 accuracy =  0.9053030303030303
--------------------------------------------------------------------------------
True:  (88, 9)
Shape:  (264, 9)
Count  7 :  (88, 9)
Count  6 :  (88, 9)
Count  5 :  (48, 9)
Count  2 :  (6, 9)
Count  3 :  (4, 9)
Count  4 :  (15, 9)
Count  1 :  (14, 9)
Count  8 :  (1, 9)
Valid user =  7 , Extracted user =  6 accuracy =  0.8787878787878788
--------------------------------------------------------------------------------
True:  (88, 9)
Shape:  (264, 9)
Count  7 :  (88, 9)
Count  8 :  (88, 9)
Count  5 :  (45, 9)
Count  1 :  (18, 9)
Count  6 :  (1, 9)
Count  2 :  (7, 9)
Count  4 :  (15, 9)
Count  3 :  (2, 9)
Valid user =  7 , Extracted user =  8 accuracy =  0.8787878787878788
--------------------------------------------------------------------------------
Valid User:  8
--------------------------------------------------------------------------------
True:  (113, 9)
Shape:  (339, 9)
Count  8 :  (113, 9)
Count  1 :  (113, 9)
C

Valid user =  2 , Extracted user =  8 accuracy =  0.8784570596797671
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
True:  (191, 9)
Shape:  (573, 9)
Count  3 :  (191, 9)
Count  1 :  (191, 9)
Count  4 :  (35, 9)
Count  5 :  (131, 9)
Count  8 :  (4, 9)
Count  2 :  (17, 9)
Count  6 :  (2, 9)
Count  7 :  (2, 9)
Valid user =  3 , Extracted user =  1 accuracy =  0.7085514834205934
--------------------------------------------------------------------------------
True:  (191, 9)
Shape:  (573, 9)
Count  3 :  (191, 9)
Count  2 :  (191, 9)
Count  5 :  (106, 9)
Count  1 :  (48, 9)
Count  4 :  (35, 9)
Count  8 :  (2, 9)
Valid user =  3 , Extracted user =  2 accuracy =  0.699825479930192
--------------------------------------------------------------------------------
True:  (191, 9)
Shape:  (573, 9)
Count  3 :  (191, 9)
Count  4 :  (191, 9)
Count  5 :  (112, 9)
Count  1 : 

...

True:  (13, 12)
Shape:  (39, 12)
Count  6 :  (13, 12)
Count  2 :  (13, 12)
Count  1 :  (5, 12)
Count  4 :  (2, 12)
Count  5 :  (5, 12)
Count  7 :  (1, 12)
Valid user =  6 , Extracted user =  2 accuracy =  0.6666666666666666
--------------------------------------------------------------------------------
True:  (405, 12)
Shape:  (1215, 12)
Count  6 :  (405, 12)
Count  3 :  (405, 12)
Count  5 :  (178, 12)
Count  1 :  (100, 12)
Count  4 :  (106, 12)
Count  7 :  (20, 12)
Count  8 :  (1, 12)
Valid user =  6 , Extracted user =  3 accuracy =  0.68559670781893
--------------------------------------------------------------------------------
True:  (405, 12)
Shape:  (1215, 12)
Count  6 :  (405, 12)
Count  4 :  (405, 12)
Count  5 :  (227, 12)
Count  3 :  (32, 12)
Count  1 :  (111, 12)
Count  7 :  (33, 12)
Count  8 :  (1, 12)
Count  2 :  (1, 12)
Valid user =  6 , Extracted user =  4 accuracy =  0.6864197530864198
--------------------------------------------------------------------------------
True

Valid user =  1 , Extracted user =  6 accuracy =  0.686183395291202
--------------------------------------------------------------------------------
True:  (1076, 14)
Shape:  (3228, 14)
Count  1 :  (1076, 14)
Count  7 :  (1076, 14)
Count  5 :  (305, 14)
Count  6 :  (313, 14)
Count  2 :  (141, 14)
Count  4 :  (174, 14)
Count  3 :  (131, 14)
Count  8 :  (12, 14)
Valid user =  1 , Extracted user =  7 accuracy =  0.6830855018587361
--------------------------------------------------------------------------------
True:  (721, 14)
Shape:  (2163, 14)
Count  1 :  (721, 14)
Count  8 :  (721, 14)
Count  2 :  (90, 14)
Count  6 :  (182, 14)
Count  3 :  (72, 14)
Count  5 :  (220, 14)
Count  4 :  (114, 14)
Count  7 :  (43, 14)
Valid user =  1 , Extracted user =  8 accuracy =  0.686084142394822
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
True:  (1911, 14)
Shape:  (5733, 

Valid user =  4 , Extracted user =  7 accuracy =  0.7753940455341506
--------------------------------------------------------------------------------
True:  (721, 14)
Shape:  (2163, 14)
Count  4 :  (721, 14)
Count  8 :  (721, 14)
Count  6 :  (213, 14)
Count  5 :  (231, 14)
Count  3 :  (85, 14)
Count  2 :  (87, 14)
Count  1 :  (66, 14)
Count  7 :  (39, 14)
Valid user =  4 , Extracted user =  8 accuracy =  0.6957928802588996
--------------------------------------------------------------------------------
Valid User:  5
--------------------------------------------------------------------------------
True:  (4244, 14)
Shape:  (12732, 14)
Count  5 :  (4244, 14)
Count  1 :  (4244, 14)
Count  6 :  (1587, 14)
Count  3 :  (639, 14)
Count  4 :  (881, 14)
Count  7 :  (314, 14)
Count  8 :  (71, 14)
Count  2 :  (752, 14)
Valid user =  5 , Extracted user =  1 accuracy =  0.6919572730128809
--------------------------------------------------------------------------------
True:  (4244, 14)
Shape:  (127

Valid user =  7 , Extracted user =  8 accuracy =  0.9639389736477115
--------------------------------------------------------------------------------
Valid User:  8
--------------------------------------------------------------------------------
True:  (179, 14)
Shape:  (537, 14)
Count  8 :  (179, 14)
Count  1 :  (179, 14)
Count  5 :  (61, 14)
Count  2 :  (25, 14)
Count  6 :  (44, 14)
Count  7 :  (6, 14)
Count  4 :  (23, 14)
Count  3 :  (20, 14)
Valid user =  8 , Extracted user =  1 accuracy =  0.6666666666666666
--------------------------------------------------------------------------------
True:  (179, 14)
Shape:  (537, 14)
Count  8 :  (179, 14)
Count  2 :  (179, 14)
Count  6 :  (45, 14)
Count  5 :  (58, 14)
Count  7 :  (9, 14)
Count  4 :  (35, 14)
Count  3 :  (20, 14)
Count  1 :  (12, 14)
Valid user =  8 , Extracted user =  2 accuracy =  0.6666666666666666
--------------------------------------------------------------------------------
True:  (179, 14)
Shape:  (537, 14)
Count  8 : 

Valid user =  3 , Extracted user =  1 accuracy =  0.8102131081624447
--------------------------------------------------------------------------------
True:  (1658, 14)
Shape:  (4974, 14)
Count  3 :  (1658, 14)
Count  2 :  (1658, 14)
Count  5 :  (568, 14)
Count  6 :  (536, 14)
Count  4 :  (276, 14)
Count  7 :  (105, 14)
Count  1 :  (151, 14)
Count  8 :  (22, 14)
Valid user =  3 , Extracted user =  2 accuracy =  0.8232810615199035
--------------------------------------------------------------------------------
True:  (1658, 14)
Shape:  (4974, 14)
Count  3 :  (1658, 14)
Count  4 :  (1658, 14)
Count  6 :  (532, 14)
Count  5 :  (591, 14)
Count  2 :  (271, 14)
Count  7 :  (99, 14)
Count  1 :  (144, 14)
Count  8 :  (21, 14)
Valid user =  3 , Extracted user =  4 accuracy =  0.8377563329312424
--------------------------------------------------------------------------------
True:  (1658, 14)
Shape:  (4974, 14)
Count  3 :  (1658, 14)
Count  5 :  (1658, 14)
Count  4 :  (376, 14)
Count  6 :  (637, 